# ATTACKS

In [2]:
from tensordict import TensorDict
from tensordict import MemoryMappedTensor as MMT

In [1]:
import torchattacks

In [1]:
# python stuff

from pathlib import Path as Path
from numpy.random import randint

# Our stuff
from datasets.cifar import Cifar
from models.model_wrap import ModelWrap

# from credibility import get_credibility

# torch stuff
import torch
from torchvision.models import vgg16, VGG16_Weights
from peepholes.peepholes import Peepholes
from peepholes.svd_peepholes import peep_matrices_from_svds as parser_fn
from credibility.DkNN import NearestNeighbor, DkNN


use_cuda = torch.cuda.is_available()
cuda_index = torch.cuda.device_count() - 3
device = torch.device(f"cuda:{cuda_index}" if use_cuda else "cpu")
print(f"Using {device} device")

#--------------------------------
# Dataset 
#--------------------------------
# model parameters
dataset = 'CIFAR100' 
seed = 29
bs = 64
data_path = '/srv/newpenny/XAI/LM/data/CIFAR100'

ds = Cifar(dataset=dataset, data_path=data_path)
ds.load_data(
        batch_size = bs,
        data_kwargs = {'num_workers': 4, 'pin_memory': True},
        seed = seed,
        )

Using cuda:5 device
dataset: CIFAR100
Files already downloaded and verified
Files already downloaded and verified


{'train': <torch.utils.data.dataloader.DataLoader at 0x7f1155fefe60>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f11560f1790>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f115df98aa0>}

In [3]:
#--------------------------------
# Model 
#--------------------------------
pretrained = True
model_dir = '/srv/newpenny/XAI/LM/models'
model_name = f'vgg16_pretrained={pretrained}_dataset={dataset}-'\
f'augmented_policy=CIFAR10_bs={bs}_seed={seed}.pth'

nn = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
in_features = 4096
num_classes = len(ds.get_classes()) 
nn.classifier[-1] = torch.nn.Linear(in_features, num_classes)
model = ModelWrap(device=device)
model.set_model(model=nn, path=model_dir, name=model_name, verbose=True)
              
model.set_target_layers(target_layers=layers_dict, verbose=True)
print('target layers: ', model.get_target_layers()) 

direction = {'save_input':True, 'save_output':False}
model.add_hooks(**direction, verbose=False) 

dry_img, _ = ds._train_ds.dataset[0]
dry_img = dry_img.reshape((1,)+dry_img.shape)
model.dry_run(x=dry_img)

loaders = ds.get_dataset_loaders()

/home/lorenzocapelli/repos/XAI/src/models/model_wrap.py:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._checkpoint = torch.load(file, map_location=self.device)



-----------------
checkpoint
-----------------
state_dict keys: 
 odict_keys(['features.0.weight', 'features.0.bias', 'features.2.weight', 'features.2.bias', 'features.5.weight', 'features.5.bias', 'features.7.weight', 'features.7.bias', 'features.10.weight', 'features.10.bias', 'features.12.weight', 'features.12.bias', 'features.14.weight', 'features.14.bias', 'features.17.weight', 'features.17.bias', 'features.19.weight', 'features.19.bias', 'features.21.weight', 'features.21.bias', 'features.24.weight', 'features.24.bias', 'features.26.weight', 'features.26.bias', 'features.28.weight', 'features.28.bias', 'classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.6.weight', 'classifier.6.bias']) 

train_loss 1.4151224618911744
val_loss 0.8791644280883157
train_accuracy 62.295
val_accuracy 74.96000000000001
epoch 59
batch_size 64
lr 0.001
-----------------

target layers:  {'classifier.0': Linear(in_features=25088, out_features=4096, bias=Tru

## BIM

In [ ]:
nb_classes = ds.config['num_classes']

In [ ]:
attack = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=10)
attack_TensorDict = TensorDict()
for ds_key, dataloader in ds.items():
    n_samples = len(dataloader.dataset)
    portion_TensorDict = TensorDict(batch_size=n_samples)
    portion_TensorDict['image'] = MMT.empty(shape=torch.Size((n_samples,)))
    portion_TensorDict['label'] = MMT.empty(shape=torch.Size((n_samples,)))
    portion_TensorDict['output'] = MMT.empty(shape=torch.Size((n_samples,)+(nb_classes,)))
    portion_TensorDict['attack_succes'] = MMT.empty(shape=torch.Size((n_samples,)))
    for data in dataloader:
        adv_images = attack(images, labels)
        